<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_04/class_004_image_processing_smart_trash_bin_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage
import tensorflow as tf

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:01<00:00, 23.2MB/s]


In [4]:
FILE='trash_nov_22_2018.zip'

In [5]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [6]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [7]:
Img_Size = 200
ref = 'jpg'
notref = 'met'

In [8]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    X.append(cv2.imread(name,0))
m = len(Y)
print(n,m)

2527 2527


In [9]:
def properties(glcm_names,img,Img_Size):
  glcm =[]
  resized = cv2.resize(img, (Img_Size,Img_Size))
  m = skimage.exposure.rescale_intensity(resized, in_range='image', out_range='dtype')
  g = skimage.feature.greycomatrix(m, [1, 2], [0, np.pi/2], levels=256,normed=True, symmetric=True)
  for x in glcm_names:
    prop = skimage.feature.greycoprops(g, x)
    glcm.append(np.max(prop))
  
  return glcm

glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

# see https://github.com/ucfilho/ANN/blob/master/ANN_2022/Image/class_001_image_processing_smart_trash_bin_012.ipynb
features =[]
for i in range(len(img_name)):
  glcm = []
  img = X[i]
  glcm = properties(glcm_names,img,Img_Size)
  
  img = cv2.imread(img_name[i])
  green_img = img[:,:,1]
  values_green = properties(glcm_names,green_img,Img_Size)

  red_img = img[:,:,0]
  values_red = properties(glcm_names,red_img,Img_Size)

  blue_img = img[:,:,2]
  values_blue = properties(glcm_names,blue_img,Img_Size)

  total= [*glcm,*values_red, *values_green, *values_blue]
  
  features.append(total)



In [10]:
lista = []
glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
names = glcm_names
for x in glcm_names:
  lista.append(x+'_r')
names = glcm_names + lista

lista = []
for x in glcm_names:
  lista.append(x+'_g')
names = names + lista

lista = []
for x in glcm_names:
  lista.append(x+'_b')
names = names + lista

print(names)

df = pd.DataFrame(features,columns=names)

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM', 'contrast_r', 'dissimilarity_r', 'homogeneity_r', 'energy_r', 'correlation_r', 'ASM_r', 'contrast_g', 'dissimilarity_g', 'homogeneity_g', 'energy_g', 'correlation_g', 'ASM_g', 'contrast_b', 'dissimilarity_b', 'homogeneity_b', 'energy_b', 'correlation_b', 'ASM_b']


In [11]:
glcm_names

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

In [12]:
n =len(img_name)
tipos = []
selected = 'find'
for x in img_name:
  result =''.join([i for i in x[:-4] if not i.isdigit()])
  if(result != selected):
    selected = result
    tipos.append(selected)


In [13]:
label =[]
for name in img_name:
  for idx, x in enumerate(tipos):
    if(x in name):
      label.append(idx)

In [14]:
df['target']=label

In [15]:
for i in range(6): 
  df_plot= df[(df["target"] == i)]
  n= df_plot.shape[0]
  print(tipos[i],n)

cardboard 403
glass 501
metal 410
paper 594
plastic 482
trash 137


In [16]:
y=df['target']
X=df.drop(['target'], axis='columns')

In [17]:
y = pd.get_dummies(y)
# check https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
y

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
2522,0,0,0,0,0,1
2523,0,0,0,0,0,1
2524,0,0,0,0,0,1
2525,0,0,0,0,0,1


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=1)

In [19]:
scaler = StandardScaler()
# transform data
scaler_X = scaler.fit(X_train)
X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)

In [20]:
nin = X_train.shape[1]
nout =y.shape[1]
nhidden = 30

In [21]:
# Create a `Sequential` model and add a Dense layer as the first layer.
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(nin,)))
model.add(tf.keras.layers.Dense(nhidden, activation='relu'))
# Now the model will take as input arrays of shape (None, 16)
# and output arrays of shape (None, 32).
# Note that after the first layer, you don't need to specify
# the size of the input anymore:
model.add(tf.keras.layers.Dense(nout, activation='softmax'))


In [22]:
X_test = np.array(X_test )
y_test = np.array(y_test)
y_test = y_test.astype(np.int32)
y_train = y_train.astype(np.int32)
X_train = X_train.reshape(-1, nin) 
X_test = X_test.reshape(-1, nin)

In [23]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [24]:
# gives us back a <keras.callbacks.History object at 0x112e61a90>
#r = model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=15, batch_size=32)
r = model.fit(X_train, y_train, epochs=1000)
print("Returned:", r)

Epoch 1/1000
53/53 [==============================] - 1s 2ms/step - loss: 1.7220 - accuracy: 0.2965
Epoch 2/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.4879 - accuracy: 0.4330
Epoch 3/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.4227 - accuracy: 0.4572
Epoch 4/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3834 - accuracy: 0.4584
Epoch 5/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3584 - accuracy: 0.4660
Epoch 6/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3368 - accuracy: 0.4773
Epoch 7/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3197 - accuracy: 0.4838
Epoch 8/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3052 - accuracy: 0.4826
Epoch 9/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2903 - accuracy: 0.4914
Epoch 10/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2793 - accuracy: 0.4938

In [25]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=model.predict(X_train)
y_calc_test=model.predict(X_test)

In [26]:
y_calc_test

array([[1.20031856e-01, 8.53445660e-03, 1.94965163e-04, 8.69788826e-01,
        1.08087424e-03, 3.68986599e-04],
       [1.14118168e-03, 8.56426120e-01, 1.42522082e-02, 7.55968764e-02,
        3.74290310e-02, 1.51545582e-02],
       [2.32029683e-03, 5.78715764e-02, 2.30475012e-02, 1.61171071e-02,
        8.79711688e-01, 2.09317617e-02],
       ...,
       [4.97977138e-02, 2.33748883e-01, 4.19269741e-01, 1.11577390e-02,
        1.46195173e-01, 1.39830813e-01],
       [1.72897169e-04, 6.18696261e-10, 6.86511444e-03, 9.86231744e-01,
        6.21343279e-05, 6.66825380e-03],
       [4.49068705e-03, 5.71154177e-01, 1.64819241e-03, 2.71167994e-01,
        1.24155089e-01, 2.73839645e-02]], dtype=float32)

In [27]:
def retorna(y):
  resp = []
  for yi in y:
    vmax =max(yi)
    for idx, x in enumerate(yi):
      if(x == vmax):
        resp.append(idx)
  return resp

In [28]:
y_test[0]

array([0, 0, 0, 0, 1, 0], dtype=int32)

In [29]:
pd.__version__

'1.3.5'

In [30]:

y_calc = retorna(y_calc_test)
y_test = retorna(y_test)

In [31]:
confusion_matrix(y_test, y_calc)

array([[106,   9,   2,  13,   2,   1],
       [  2,  94,  28,  15,  26,   1],
       [  2,  29,  72,  14,  11,   7],
       [  7,  15,  15, 129,  27,   3],
       [  6,  25,   7,  18, 101,   2],
       [  1,  10,  13,   6,   6,   9]])

In [32]:
# Classification Report
print('\n\n', classification_report(y_test, y_calc, target_names=tipos))



               precision    recall  f1-score   support

   cardboard       0.85      0.80      0.82       133
       glass       0.52      0.57      0.54       166
       metal       0.53      0.53      0.53       135
       paper       0.66      0.66      0.66       196
     plastic       0.58      0.64      0.61       159
       trash       0.39      0.20      0.26        45

    accuracy                           0.61       834
   macro avg       0.59      0.56      0.57       834
weighted avg       0.61      0.61      0.61       834



In [33]:
# Classification Report
print('\n\n', classification_report(y_test, y_test, target_names=tipos))



               precision    recall  f1-score   support

   cardboard       1.00      1.00      1.00       133
       glass       1.00      1.00      1.00       166
       metal       1.00      1.00      1.00       135
       paper       1.00      1.00      1.00       196
     plastic       1.00      1.00      1.00       159
       trash       1.00      1.00      1.00        45

    accuracy                           1.00       834
   macro avg       1.00      1.00      1.00       834
weighted avg       1.00      1.00      1.00       834

